In [13]:
import os
from os.path import join
from scipy.io import loadmat
import glob
import pickle
import numpy as np
from antropy import lziv_complexity
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from mne import viz, stats
from matplotlib import pyplot as plt

In [15]:
data_dir = "../transformer/data/200/"
result_dir = "../results/lziv"

stages = ["AWA", "NREM", "REM"]
conditions = ["CAF", "PLAC"]

In [3]:
def get_paths(root, subject="*", stage="*", condition="*"):
    return glob.glob(join(root, f"{subject}n*_{stage}_*_{condition}.npy"))

In [4]:
def complexity(epoch):
    return lziv_complexity((epoch > np.median(epoch)).astype(int), normalize=True)

result = {}
for stage in stages:
    result[stage] = {}
    for condition in conditions:
        print(f"processing {stage} {condition}:")
        # load data
        paths = get_paths(data_dir, stage=stage, condition=condition)
        
        data = {}
        for path in tqdm(paths, desc="loading data"):
            subject = path.split(os.sep)[-1].split("_")[0].split("n")[0]
            if subject in data:
                data[subject] = np.concatenate([data[subject], np.load(path)])
            else:
                data[subject] = np.load(path)
        
        result[stage][condition] = []
        for subject in tqdm(data.keys(), desc="estimating complexity"):
            dat = data[subject].transpose((0, 2, 1)).reshape(-1, 5120)
            res = Parallel(n_jobs=-1)(delayed(complexity)(epoch) for epoch in dat)
            result[stage][condition].append(np.array(res).reshape(-1, 20).mean(axis=0))
        result[stage][condition] = np.stack(result[stage][condition])
        
    result[stage] = stats.permutation_t_test(result[stage]["CAF"] - result[stage]["PLAC"],
                                             n_permutations=1000, tail=0, n_jobs=-1)

processing AWA CAF:


loading data:   0%|          | 0/21 [00:00<?, ?it/s]

estimating complexity:   0%|          | 0/8 [00:00<?, ?it/s]

processing AWA PLAC:


loading data:   0%|          | 0/14 [00:00<?, ?it/s]

...

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.8s remaining:    3.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.9s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.0s finished


processing NREM CAF:


loading data:   0%|          | 0/95 [00:00<?, ?it/s]

estimating complexity:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
with open(join(result_dir, "lziv.pkl"), "wb") as f:
    pickle.dump(result, f)

In [ ]:
# load EEG sensor positions to be used in the topomaps
sensor_pos = loadmat(join("../data", 'Coo_caf'))['Cor'].T
sensor_pos = np.array([sensor_pos[1], sensor_pos[0]]).T

In [ ]:
vmin = min(val[0].min() for val in result.values())
vmax = max(val[0].max() for val in result.values())

fig, axes = plt.subplots(ncols=3, figsize=(15, 5))
for stage, ax in zip(stages, axes):
    curr_t = result[stage][0]
    curr_p = result[stage][1]
    mask = curr_p < 0.05
        
    ax.set_title(stage)
    viz.plot_topomap(curr_t, sensor_pos, mask=mask, vmin=vmin, vmax=vmax, contours=False, axes=ax, show=False)